In [1]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
import cv2
from PyQt5 import QtCore
from PyQt5 import QtWidgets
import sip

# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

# in case you are in debug mode
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

# use this function to create a Viewer, solves the problem of window not raising
viewer=None
def CreateViewer():
    global viewer
    viewer=PyViewer()
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    viewer_py.setVisible(True)
    viewer_py.show()
    viewer_py.setFocus()
    viewer_py.showMaximized()
    viewer_py.activateWindow()
    return viewer

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets']
Starting OpenVisus C:\Users\Vlaerio's PC\AppData\Roaming\Python\Python37\site-packages\OpenVisus\__init__.py 3.7.9 (tags/v3.7.9:13c94747c7, Aug 17 2020, 18:58:18) [MSC v.1900 64 bit (AMD64)] sys.version_info(major=3, minor=7, micro=9, releaselevel='final', serial=0) ...
QT_PLUGIN_PATH C:\Users\Vlaerio's PC\AppData\Roaming\Python\Python37\site-packages\PyQt5\Qt5/plugins


In [2]:
# do not change this cell for very good reasons (otherwise Qt5 loop won't work)
# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
import time
time.sleep(2)

# Define some utilities to perform viewer animations

In [3]:
# //////////////////////////////////////////////////////////////
class RunActions:
    
    def __init__(self):
        self.v=[]
        self.cursor=0
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.onTimer)
        
    def addAction(self, fn,**kwargs):
        self.v.append(lambda : fn(**kwargs))
        
    def addSleep(self,msec):
        self.v.append(msec)
        
    def onTimer(self):
        
        # if the last action was an action and the viewer is still running, I need to wait
        if self.cursor>0 and callable(self.v[self.cursor-1]) and viewer.isRunning():
            return
        
        self.timer.stop()
        
        if self.cursor>= len(self.v):
            return
        
        cur=self.v[self.cursor]
        self.cursor+=1
        
        # is a delay?
        if isinstance(cur,int):
            print("*** Sleep",cur)
            self.timer.start(cur) 
            
        # is a real action, run it and later wait for completition
        elif callable(cur):
            cur()
            self.timer.start(20) 
            
        # there must be a problem
        else:
            raise Exception("internal error")
   
    def start(self): 
        self.timer.start(1) 
        
    def stop(self):
        self.timer.stop()
        
print("Utilities defined")

Utilities defined


Some predefined actions

In [4]:
def RotateScene(axis=(0,0,1),angle=10):
    print("*** RotateScene",axis,angle)
    glcamera=viewer.getGLCamera()
    viewer.getGLCamera().setRotation(glcamera.getRotation() * Quaternion(Point3d(axis),math.radians(angle)))
    viewer.refreshAll()
    viewer.postRedisplay()
    
def TakeSnapshot(filename="temp.png",):
    print("*** TakeSnapshotAction",filename)  
    # viewer.takeSnapshot(False,filename) BROKEN
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    screenshot = QtWidgets.QApplication.primaryScreen().grabWindow(viewer_py.winId() )
    screenshot.save(filename)
    
def OpenScene(filename=""):
    print("*** OpenScene",filename)
    viewer.open(filename)
    
def DropSelection():
    print("*** DropSelection")
    viewer.dropSelection()
    
def HideDatasetBounds(uuid="dataset"):
    print("*** HideDatasetBounds",uuid)
    # **** make sure that the dataset node has UUID `dataset` (you can save an xml and inspect it) ***
    dataset_node=DatasetNode.castFrom(viewer.findNodeByUUID(uuid))
    if dataset_node:
        dataset_node.setShowBounds(False)
    else:
        print("Failed to find a node with uuid",uuid)

# Create scenes

In [ ]:

filename_src=r"D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-0002.xml"
filename_dist_prefix=r"D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame"
frame = 0 
for alpha in range(00,1510,50):
#    actions=RunActions()
    filename_dist = filename_dist_prefix +  ".{:03d}.xml".format(frame)
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ", alpha, " <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    #input file
    fin = open(filename_src, "rt")
    #output file to write the result to
    fout = open(filename_dist, "wt")
    #for each line in the input file
    for line in fin:
        #read replace the string and write to output file
        dist = 146
        line1 = line.replace("146",str(dist+alpha))
        dirst = 375
        fout.write(line1.replace("375",str(dist+alpha)))
    #close input and output files
    fin.close()
    fout.close()
    frame = frame +1


# Create Frames

In [ ]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [ ]:
snapshots=sorted(glob.glob(r"D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame*xml"))

# limit for debugging...
#snapshots=snapshots[:4] 

actions=RunActions()
for I,snapshot in enumerate(snapshots):
    print("\t",I,snapshot)
    actions.addAction(OpenScene,filename=snapshot)
    actions.addAction(DropSelection)
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset1")
    actions.addAction(HideDatasetBounds,uuid="modelview2")
    actions.addAction(HideDatasetBounds,uuid="modleview1")
    actions.addAction(HideDatasetBounds,uuid="glcamera")
    actions.addAction(HideDatasetBounds,uuid="dataset_time")
    actions.addAction(HideDatasetBounds,uuid="slice")
    actions.addAction(HideDatasetBounds,uuid="slice3")
    actions.addAction(HideDatasetBounds,uuid="volume")
    actions.addAction(HideDatasetBounds,uuid="modelview")
    actions.addAction(HideDatasetBounds,uuid="isocontour1")
    actions.addAction(HideDatasetBounds,uuid="slice_field")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(TakeSnapshot,filename=os.path.splitext(snapshot)[0] + ".png")
    
    # you can add some sleep between actions but it's not necessary
    # automatically it can wait for the viewer to become idle
    actions.addSleep(15000) 
    
actions.start()

# Recreate Individual Frames

In [21]:
# create single frame
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

In [22]:
snapshots=sorted(glob.glob(r"D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame.026.xml"))

# limit for debugging...
#snapshots=snapshots[:4] 

actions=RunActions()
for I,snapshot in enumerate(snapshots):
    print("\t",I,snapshot)
    actions.addAction(OpenScene,filename=snapshot)
    actions.addAction(DropSelection)
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset1")
    actions.addAction(HideDatasetBounds,uuid="modelview2")
    actions.addAction(HideDatasetBounds,uuid="modleview1")
    actions.addAction(HideDatasetBounds,uuid="glcamera")
    actions.addAction(HideDatasetBounds,uuid="dataset_time")
    actions.addAction(HideDatasetBounds,uuid="slice")
    actions.addAction(HideDatasetBounds,uuid="slice3")
    actions.addAction(HideDatasetBounds,uuid="volume")
    actions.addAction(HideDatasetBounds,uuid="modelview")
    actions.addAction(HideDatasetBounds,uuid="isocontour1")
    actions.addAction(HideDatasetBounds,uuid="slice_field")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(HideDatasetBounds,uuid="dataset")
    actions.addAction(TakeSnapshot,filename=os.path.splitext(snapshot)[0] + ".png")
    
    # you can add some sleep between actions but it's not necessary
    # automatically it can wait for the viewer to become idle
    actions.addSleep(15000) 
    
actions.start()

	 0 D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame.026.xml
*** OpenScene D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame.026.xml
*** DropSelection
2022-04-30 23:23:05.814388 PyScriptingNode Got in input (1, 63, 49) uint8 origin Slice 2022-04-30 23:23:05.817388 PyScriptingNode Got in input (28, 32, 63) uint8 origin IsoContour 

2022-04-30 23:23:05.820388 PyScriptingNode Got in input (1, 63, 124) uint8 origin Slice 
2022-04-30 23:23:05.822389 PyScriptingNode Output is  (1, 63, 124) uint8 msec 2 
2022-04-30 23:23:05.822889 PyScriptingNode Output is  (1, 63, 49) uint8 msec 9 
(28, 32, 63)
2022-04-30 23:23:05.827389 PyScriptingNode Output is  (28, 32, 63) float32 msec 10 
2022-04-30 23:23:06.312975 PyScriptingNode Got in input (55, 63, 125) uint8 origin IsoContour 2022-04-30 23:23:06.315475 PyScriptingNode Got in input (32, 17, 63) uint8 origin Volume 2022-04-30 23:23:06.317475 PyScriptingNode Got in input (1, 64, 128) uint8 origin Slice 

2022-04-30 23:23:06.319976 PyS

# Create the Video

Create Video

In [28]:
from moviepy.editor import *

files0 = sorted(glob.glob(r"D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame*png"))
files1 = sorted(glob.glob(r"D:\_PANIA_NEW_DATA\animation\2-datsets-side-by-side-frame*png"),reverse=True)

files = files0 + files1 + files0 + files1 + files0 + files1 
print (files1)

clip = ImageSequenceClip(files, fps = 4) 
clip.write_videofile(r"D:\_PANIA_NEW_DATA\animation\datsets-pania-side-by-side.mp4", fps = 24)

['D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.027.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.026.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.025.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.024.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.023.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.022.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.021.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.020.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.019.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.018.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.017.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.016.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.015.png', 'D:\\_PANIA_NEW_DATA\\animation\\2-datsets-side-by-side-frame.0

Moviepy - Done !
Moviepy - video ready D:\_PANIA_NEW_DATA\animation\datsets-pania-side-by-side.mp4
